In [1]:
import glob
import pandas as pd
from datetime import datetime
# import numpy as np
import plotly.express as px
# import bar_chart_race as bcr

In [4]:
semaine = 'mars23-27/'

In [5]:
all_games = glob.glob(semaine + "World*.csv")
games = pd.concat((pd.read_csv(f, usecols=[1, 2, 3, 4, 5], header=None)[[2, 3, 4, 5, 1]] for f in all_games), axis=0, ignore_index=True)
games = games.rename(columns={2:"Date", 3:"Game", 4:"Viewers", 5:"Langage", 1:"Streamer"})
print(f" ** Memory usage of the file - {sum(games.memory_usage()) * 0.000001} MB for {len(games.index)} Rows")

 ** Memory usage of the file - 1.783048 MB for 44573 Rows


In [6]:
languages = pd.read_csv('languages.csv', sep='|')

In [7]:
# map Langage values to languages names
games['Langage'] = games['Langage'].map(languages.set_index('code')['en'])

In [8]:
games['Date'] = pd.to_datetime(games['Date'])

In [9]:
games.head()

,Date,Game,Viewers,Langage,Streamer
0,2022-03-23 10:30:03,League of Legends,99111,Korean,LCK_Korea
1,2022-03-23 10:30:03,League of Legends,47888,English,LCK
2,2022-03-23 10:30:03,League of Legends,36410,Korean,울프
3,2022-03-23 10:30:03,World of Warcraft,25839,English,Echo_Esports
4,2022-03-23 10:30:03,Slots,24134,English,Trainwreckstv


- nombre de viewers par langues
- nombre de viewers par jeu
- nombre de vues totales par streamer


## BAR CHART RACE 1
dataframe de la forme:</br>
date / valeur1 / ... / valeur n

In [6]:
df = games.groupby([pd.Grouper(key='Date', axis=0, freq='H'), 'Langage']).agg('sum').reset_index()

In [7]:
df = df[['Date', 'Langage', 'Viewers']].pivot(index='Date', columns='Langage', values='Viewers')
df = df.fillna(0)

In [10]:
df.head()

Langage,Arabic,Czech,German,Greek,English,Spanish,French,Hungarian,Italian,Japanese,Korean,other,Polish,Portuguese,Russian,Thai,Turkish,Chinese
Date,,,,,,,,,,,,,,,,,,
2022-03-23 10:00:00,0.0,0.0,45500.0,0.0,693761.0,24748.0,72194.0,0.0,27667.0,100738.0,302138.0,0.0,0.0,5488.0,65218.0,2300.0,0.0,30088.0
2022-03-23 11:00:00,0.0,0.0,105343.0,0.0,1241729.0,42310.0,147060.0,0.0,48191.0,225731.0,262397.0,0.0,0.0,16165.0,163445.0,43394.0,0.0,91003.0
2022-03-23 12:00:00,0.0,0.0,116547.0,0.0,1415966.0,39506.0,144085.0,0.0,40112.0,268546.0,439006.0,0.0,0.0,43664.0,186065.0,62772.0,0.0,96575.0
2022-03-23 13:00:00,0.0,0.0,103037.0,0.0,1558476.0,55785.0,140854.0,0.0,40463.0,251330.0,468690.0,0.0,0.0,67092.0,254627.0,71763.0,0.0,123551.0
2022-03-23 14:00:00,0.0,0.0,130786.0,0.0,1653192.0,65839.0,179692.0,0.0,35832.0,220246.0,437628.0,0.0,9483.0,87305.0,248262.0,68634.0,12669.0,143121.0


In [11]:
bcr.bar_chart_race(
    df=df,
    filename=semaine+'Langage.mp4',
    orientation='h',
    sort='desc', ###
    n_bars=10, ###
    fixed_order=False, ###
    fixed_max=False,
    steps_per_period=30, ###
    interpolate_period=False,
    label_bars=True,
    bar_size=.95, ###
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='%B %d, %Hh%M', ###
    period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                      's': f'Simultaneous Viewers: {v.sum():,.0f}+', #v.nlargest(10).sum()
                                      'ha': 'right', 'size': 9},
    # perpendicular_bar_func='median',
    period_length=500, ###
    figsize=(7, 5), ###
    dpi=300, ###
    cmap='dark12',
    title='Viewers by Language',
    title_size='',
    bar_label_size=9,
    tick_label_size=8,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .9}, ###
    filter_column_colors=False)  

findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appe

In [12]:
df2 = games.groupby([pd.Grouper(key='Date', axis=0, freq='H'), 'Game']).agg('sum').reset_index()

In [13]:
df2 = df2[['Date', 'Game', 'Viewers']].pivot(index='Date', columns='Game', values='Viewers')
df2 = df2.fillna(0)

In [14]:
# filter out columns with sum < 10000
df2 = df2.loc[:, (df2.sum(axis=0) > 5000000)]
df2.head()

Game,Apex Legends,Call of Duty: Warzone,Counter-Strike: Global Offensive,Dota 2,Elden Ring,FIFA 22,Fortnite,Grand Theft Auto V,Hearthstone,Just Chatting,...,Lost Ark,Mahjong Soul,Minecraft,Rocket League,Rust,Slots,Special Events,Sports,VALORANT,World of Warcraft
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-23 10:00:00,37985.0,11380.0,15235.0,75301.0,70035.0,0.0,27227.0,19164.0,34976.0,187752.0,...,21652.0,3905.0,0.0,0.0,34581.0,53583.0,0.0,7252.0,28193.0,92953.0
2022-03-23 11:00:00,76528.0,25445.0,94043.0,173579.0,147213.0,0.0,56846.0,69469.0,72740.0,286755.0,...,42735.0,25769.0,0.0,0.0,81157.0,107776.0,0.0,23288.0,95848.0,204645.0
2022-03-23 12:00:00,88067.0,32264.0,242791.0,171597.0,162306.0,0.0,63499.0,69073.0,35306.0,388429.0,...,46426.0,37525.0,13094.0,0.0,70894.0,108545.0,0.0,32656.0,112475.0,223040.0
2022-03-23 13:00:00,78963.0,29699.0,240347.0,198321.0,215551.0,0.0,60903.0,74116.0,20612.0,407716.0,...,48073.0,115576.0,4711.0,0.0,87272.0,115087.0,0.0,19852.0,146781.0,248013.0
2022-03-23 14:00:00,90427.0,32020.0,189244.0,120721.0,230621.0,4395.0,72055.0,111502.0,21883.0,461132.0,...,43642.0,126889.0,0.0,0.0,100286.0,123604.0,0.0,0.0,191914.0,273743.0


In [15]:
bcr.bar_chart_race(
    df=df2,
    filename=semaine+'Game.mp4',
    orientation='h',
    sort='desc', ###
    # n_bars=25, ###
    fixed_order=True, ###
    fixed_max=True,
    steps_per_period=30, ###
    interpolate_period=False,
    label_bars=True,
    bar_size=.95, ###
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='%B %d, %Hh%M', ###
    period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                      's': f'Simultaneous Viewers: {v.sum():,.0f}+', #v.nlargest(10).sum()
                                      'ha': 'right', 'size': 8},
    # perpendicular_bar_func='median',
    period_length=500, ###
    figsize=(5, 5), ###
    dpi=300, ###
    cmap='dark12',
    title='Viewers by Game',
    title_size='',
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .9}, ###
    filter_column_colors=False)  

findfont: Font family ['Helvetica'] not found. Falling back to DejaVu Sans.
C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


## AVERAGE VIEWERS PER STREAMER

In [16]:
avg = games[['Streamer', 'Langage', 'Viewers']].groupby(['Streamer', 'Langage']).agg(['mean', 'count'])['Viewers']
avg = avg.reset_index()
avg['mean'] = avg['mean'].apply(lambda x : int(x))

In [17]:
avg = avg[avg['count'] >+ 4 ]

In [18]:
avg

,Streamer,Langage,mean,count
0,1PVCS,fr,4729,7
2,2wintails,en,3568,6
3,39daph,en,6378,115
5,5uppp,en,3020,8
7,8cho,es,4445,6
...,...,...,...,...
1605,플러리,ko,2817,7
1606,플레임__,ko,3506,56
1608,핑맨,ko,4172,10
1609,한갱,ko,8387,82


In [19]:
fig = px.scatter(
    avg,
    x='mean',
    y='Langage',
    log_x=True,
    color='Streamer',
    size='count',
    size_max=50,
    hover_name='Streamer',
    hover_data=['mean', 'Langage', 'count'],
    height=900,
    width=1400,
    title='Viewers per Langage per Streamer',
    template='plotly_dark',
    labels=False,
    color_discrete_sequence=px.colors.qualitative.Dark24,
)
fig.update_layout(showlegend=False)
fig.show()

In [20]:
fig = px.scatter(
    avg,
    x='mean',
    y='count',
    color='Langage',
    size='mean',
    # size_max=50,
    hover_name='Streamer',
    hover_data=['mean', 'Langage'],
    height=900,
    width=1400,
    title='Average Viewers vs Hours streamed',
    template='plotly_dark',
    labels=False,
    color_discrete_sequence=px.colors.qualitative.Dark24,
)
fig.update_layout(showlegend=False)
fig.show()

## SUNBURST JOURS => LANGUES => STREAMER (Viewers)

https://plotly.com/python/sunburst-charts/


Structure de données:</br>
["Jours de la semaine", "Langues", "Streamers"]</br>
parents = ["", "Jours de la semaine", "Langues"]</br>
values = [total views per day, total views per langage, Average viewers per streamer]</br>
on se moque que la donnée soit erronée, tant que les proportions sont respectées

In [67]:
day = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [68]:
gamesDay = games.groupby([pd.Grouper(key='Date', axis=0, freq='D'), 'Langage']).agg('sum').reset_index()

In [69]:
gamesDay['Date'] = gamesDay['Date'].apply(lambda x : day[x.weekday()])

In [71]:
gamesDay.head()

,Date,Langage,Viewers
0,Wednesday,Chinese,893404
1,Wednesday,English,32493373
2,Wednesday,French,4167967
3,Wednesday,German,4127593
4,Wednesday,Italian,1218336


In [88]:
fig = px.sunburst(
    gamesDay,
    path=["Date", "Langage"],
    values="Viewers",
    template='plotly_dark',
)
# change order to ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fig.update_layout(
    extendsunburstcolors=True,
    title_text='Viewers per Day per Langage',
    title_x=0.5,
    # title_y=0.5,
    title_font_size=20,
    title_font_family='Arial',
    title_font_color='white',
    title_xanchor='center',
    # title_yanchor='auto',
    title_xref='paper',
    title_yref='paper',
)

fig.show()

C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Et les streamers

In [168]:
streamerDay['Date'] = streamerDay['Date'].apply(lambda x : x[:-9])
streamerDay['Date'] = pd.to_datetime(streamerDay['Date'])

In [169]:
streamerDay = streamerDay.groupby(['Date', 'Langage', 'Streamer']).agg('mean')
streamerDay = streamerDay.reset_index()

In [170]:
streamerDay['Date'] = streamerDay['Date'].apply(lambda x : day[x.weekday()])

In [171]:
streamerDay

,Date,Langage,Streamer,Viewers
0,Monday,ar,ABOKYAN,1121.666667
1,Monday,ar,ABo3aBD,1110.000000
2,Monday,ar,Ahmed_show,2716.666667
3,Monday,ar,AiShowoo,1376.000000
4,Monday,ar,FRS9,1094.200000
...,...,...,...,...
15342,Sunday,zh,阿阿汪,2216.226415
15343,Sunday,zh,馬桶拉,1515.000000
15344,Sunday,zh,鳥屎,1111.666667
15345,Sunday,zh,黑大大,1135.750000


In [172]:
data = dict(
    categories=gamesDay['Date'],
    parent=["" for _ in range(len(gamesDay))],
    values=gamesDay['Viewers'],
)

In [182]:
import plotly.graph_objects as go

In [25]:
# fig = px.sunburst(
#     streamerDay,
#     path=["Date", "Langage", "Streamer"],
#     values="Viewers",
#     color="Streamer",
#     color_discrete_sequence=px.colors.qualitative.Dark24,
#     hover_data=["Viewers", "Streamer"],
#     title="Viewers per Streamer per Langage per Day",
#     template="plotly_dark",
#     labels=False,
# )
# fig.show()

In [115]:
data = dict(
    days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "fr", "en", "es", "pt", "ko", "fr", "en", "es", "pt", "ko", "fr", "en", "es", "pt", "ko", "fr", "en", "es", "pt", "ko", "fr", "en", "es", "pt", "ko", "fr", "en", "es", "pt", "ko", "fr", "en", "es", "pt", "ko", ],
    parent=["", "", "", "", "", "", "", "Monday", "Monday", "Monday", "Monday", "Monday", "Tuesday", "Tuesday", "Tuesday", "Tuesday", "Tuesday", "Wednesday", "Wednesday", "Wednesday", "Wednesday", "Wednesday", "Thursday", "Thursday", "Thursday", "Thursday", "Thursday", "Friday", "Friday", "Friday", "Friday", "Friday", "Saturday", "Saturday", "Saturday", "Saturday", "Saturday", "Sunday", "Sunday", "Sunday", "Sunday", "Sunday"],
    value=[10, 14, 12, 10, 50, 20, 18, 10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, 4, 10])

In [116]:
fig = px.sunburst(
    data,
    names='days',
    parents='parent',
    values='value',
)
fig.show()

In [ ]:
px.sunburst(
    data,
    names='days',
    parents='parent',
    values='value',
).show()
